In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import time
import sys
import resource
from torch.utils.data import DataLoader

from data_classes import *
from read_input import *
from read_trainset import *
from network import *
from prepare_batches import *
from traininit import *
from data_set import *
from data_loader import *
from optimization_step import *
from output_nn import *
from py_aeio import *

/home/riccardo/.venvs/gen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cpu"

In [4]:
tin_file = "../../../BayesianNN/AenetBayesian/train.in"
tin = read_train_in(tin_file)
torch.manual_seed(tin.pytorch_seed)
np.random.seed(tin.numpy_seed)
if tin.verbose: io_input_reading(tin)
tin.train_file = '../../../BayesianNN/AenetBayesian/data.train.ascii'
tin.train_forces_file = '../../../BayesianNN/AenetBayesian/data.train.forces'

----------------------------------------------------------------------
                       Reading input information                      
----------------------------------------------------------------------

Reading input parameters.
These are the parameters selected for training:
        - TRAININGSET: data.train.ascii
        - TESTPERCENT: 0.1
        - ITERATIONS:  5000
        - ITERWRITE:   1
        - BATCHSIZE:   128
        - MEMORY_MODE: cpu

        - FORCES:      True
        - alpha:       0.1



In [5]:
if tin.verbose : io_trainingset_information()
list_structures_energy, list_structures_forces, list_removed, max_nnb, tin = read_list_structures(tin)

----------------------------------------------------------------------
                   Reading training set information                   
----------------------------------------------------------------------

Training set information will be read now. If force training is
required this proccess may take some time.



In [6]:
N_removed = len(list_removed)
N_struc_E = len(list_structures_energy)
N_struc_F = len(list_structures_forces)
if tin.verbose : io_trainingset_information_done(tin, tin.trainset_params, N_struc_E, N_struc_F, N_removed)

if tin.verbose : io_prepare_batches()

N_batch_train, N_batch_valid = select_batch_size(tin, list_structures_energy, list_structures_forces)

# Join datasets with forces and only energies in a single torch dataset AND prepare batches
train_forces_data, valid_forces_data, train_energy_data, valid_energy_data = select_batches(tin, tin.trainset_params, device, list_structures_energy, list_structures_forces,
                                                                                        max_nnb, N_batch_train, N_batch_valid)

del list_structures_energy
del list_structures_forces

if tin.verbose : io_prepare_batches_done(tin, train_energy_data, train_forces_data)

The network output energy will be normalized to the interval [-1,1].
    Energy scaling factor:  f =     0.000502
    Atomic energy shift  :  s =  1985.644617


Number of structures in the data set:                 154
Number of structures with force information:           15

Atomic species in the training set: Pd  O

Average energy (eV/atom) :  1571.876188
Minimum energy (eV/atom) :    -4.525785
Maximum energy (eV/atom) :  3975.815020

----------------------------------------------------------------------
                    Preparing batches for training                    
----------------------------------------------------------------------

Batches for training are being prepared now. If force training is
required, this may take some time.

If the number of structures is not divisible by the batch size, the actual
batch size may be slightly changed.

Requested batch size: 128
Actual batch size   : 140
Number of batches   : 1

Energy batch size   : 126
Forces batch size   : 14



In [7]:
grouped_train_data = GroupedDataset(train_energy_data, train_forces_data,
									 memory_mode=tin.memory_mode, device=device, dataname="train")
grouped_valid_data = GroupedDataset(valid_energy_data, valid_forces_data,
									 memory_mode=tin.memory_mode, device=device, dataname="valid")

In [8]:
del train_forces_data
del valid_forces_data
del train_energy_data
del valid_energy_data

In [60]:
# 4. Initialize dataloader
grouped_train_loader = DataLoader(grouped_train_data, batch_size=1, shuffle=False,
                                  collate_fn=custom_collate, num_workers=0)
grouped_valid_loader = DataLoader(grouped_valid_data, batch_size=1, shuffle=False,
                                  collate_fn=custom_collate, num_workers=0)

In [61]:
for data_batch in grouped_train_loader:
    grp_descrp, grp_energy, logic_reduce = data_batch[0][10], data_batch[0][11], data_batch[0][12]
    print('ciao')

grp_descrp[0] = grp_descrp[0].float()
grp_descrp[1] = grp_descrp[1].float()

logic_reduce[0] = logic_reduce[0].float()
logic_reduce[1] = logic_reduce[1].float()

ciao


In [51]:
from bayesian_network import BayesianNetAtoms
from pyro.nn.module import to_pyro_module_
from pyro.nn import PyroSample
import pyro.distributions as dist
from pyro.infer.autoguide import AutoDiagonalNormal
import pyro

model = NetAtom(tin.networks_param["input_size"], tin.networks_param["hidden_size"],
			    tin.sys_species, tin.networks_param["activations"], tin.alpha, device)


to_pyro_module_(model)
# assert isinstance(model, PyroModule[nn.Sequential])
# assert isinstance(model[0], PyroModule[nn.Linear])

# Now we can attempt to be fully Bayesian:
for m in model.modules():
    for name, value in list(m.named_parameters(recurse=False)):
        if name == 'weight':
            setattr(m, name, PyroSample(prior=dist.Normal(0, 1)
                                              .expand(value.shape)
                                              .to_event(value.dim())))
        if name == 'bias':
            setattr(m, name, PyroSample(prior=dist.Normal(0, 1)
                                              .expand(value.shape)
                                              .to_event(value.dim())))
                                              
def bnn(grp_descrp, logic_reduce, grp_energy=None):
    mean = model.forward(grp_descrp, logic_reduce)
    sigma = pyro.sample('noise', dist.Uniform(0,10))
    with pyro.plate('data', len(logic_reduce[0])):
        pyro.sample('obs', dist.Normal(mean, sigma), obs=grp_energy)
    return mean

# model = BayesianNetAtoms(net)       
guide = AutoDiagonalNormal(model)

In [52]:
from pyro.infer import SVI, Trace_ELBO

pyro.clear_param_store()
adam = pyro.optim.Adam({"lr": 0.05})
svi = SVI(bnn, guide, adam, loss=Trace_ELBO())
for x in range(100):
    if x % 1000:
        print(svi.step(grp_descrp, logic_reduce))

/home/riccardo/.venvs/gen/lib/python3.10/site-packages/pyro/util.py:303: UserWarning: Found vars in model but not guide: {'obs', 'noise'}
  warnings.warn(f"Found vars in model but not guide: {bad_sites}")


1127.2491780257124
998.0746624780526
1062.641128716384
1006.7460678097312
1102.9588505370573
770.9377683545463
901.5406620508594
997.3241370895846
958.104354207214
906.0995219275517
925.4997862640483
955.5460794813882
773.0798004979644
872.2207620628067
774.5487358834171
402.24532698187755
450.3652179571744
831.5771964440033
656.0656835064688
728.116922984229
823.7300865913305
611.1633742005722
788.4532996271396
442.8920322990176
749.3957118298707
691.5676479986117
358.07228713896035
557.8536162900093
503.46395584586185
344.8042948321847
555.8524172683201
656.0074419104378
563.6517049133467
566.7245855306235
447.4464929380574
592.265604650533
481.8433150252247
579.011279139622
559.5050316116208
385.8000432820786
189.1230016240923
433.64359185289106
402.74876451080456
504.0882979524413
495.7851158702797
495.72588445398924
410.3812660911774
474.11985443937976
282.749533998781
458.0088625785587
431.7557110175327
280.5311697599136
315.52163870781203
214.23835241037864
463.5048912648127
89.

In [54]:
from pyro.infer import Predictive

def summary(samples):
    site_stats = {}
    for k, v in samples.items():
        site_stats[k] = {
            "mean": torch.mean(v, 0).numpy(),
            "std": torch.std(v, 0).numpy(),
            "5%": v.kthvalue(int(len(v) * 0.05), dim=0)[0].numpy(),
            "95%": v.kthvalue(int(len(v) * 0.95), dim=0)[0].numpy(),
        }
    return site_stats


predictive = Predictive(bnn, guide=guide, num_samples=800,
                        return_sites=("linear.weight", "obs", "_RETURN"))

samples = predictive(grp_descrp, logic_reduce)
mu = summary(samples)

In [56]:
grp_energy

tensor([ 107.8656,  -62.9981,  107.8850,  107.8829,  107.9120,  107.8639,
         107.8646,  107.8906,  -99.9961,  -99.9691,  107.8644,  107.8754,
         -99.9705,  107.8649,  107.8714,  -99.9766,  107.9019,  -99.9954,
         -99.9980,  -99.6753,  -97.9963,  -62.9968,  107.8641,  107.8646,
         -62.9980,  107.8661,  -99.9992,  -99.8661,  107.8647,  -97.9963,
         -62.9973,  107.8651,  107.8647,  -99.9891,  107.8643,  -99.8998,
         107.9153,  -62.9980,  -99.9987,  -99.9929,  107.9058,  -99.9681,
         -62.9978,  -99.8867,  -62.9929,  -62.9979,  -99.9996,  -62.9978,
         107.8644,  107.8651,  107.8946,  -99.8372,  -62.9986,  107.8798,
         107.8644,  -99.9973,  -99.9800,  -99.9836,  -99.9756,  -99.9650,
         -99.9569,  -99.9695,  -99.9861,  107.8643,  -62.9951,  -99.9854,
         -97.9964,  -99.9997,  107.8879,  107.8647,  107.9094,  -99.9964,
         -62.9978,  107.8645,  -99.9558,  -97.9962,  -99.9997,  107.8785,
         -99.9827,  -99.9998,  -99.999

In [57]:
mu['obs']

{'mean': array([ 2.3848079 , 21.66399528,  5.76464058,  4.16636907,  1.42449248,
         2.74040843,  2.58288072,  2.17378361, 40.95036338, 48.3928833 ,
         3.08993269,  1.13196029, 40.45157068,  3.73691255,  2.10344371,
        37.41647122,  7.69162924, 39.15182134, 39.61502195, 26.59530364,
        39.09463355, 26.14001988,  3.05205677,  3.4839045 , 25.93950344,
         3.28807881, 37.65493851, 26.22537251,  2.90429909, 39.46207201,
        25.58188102,  3.78700873,  2.31009521, 48.99096537,  3.85123192,
        39.59815541,  5.18729516, 25.84632715, 37.58234979, 41.74626509,
         8.69878039, 32.92982649, 25.97981655, 24.97186323, 22.76190847,
        25.8170111 , 38.21579378, 24.98758225,  2.93948307,  3.18215035,
         6.4882991 , 28.89540269, 26.06465003,  8.67426575,  3.38916777,
        39.69315361, 35.5826221 , 30.80202468, 37.27041395, 37.88483327,
        43.82434932, 41.03512679, 47.14802515,  2.87794924, 25.3679537 ,
        48.66029678, 39.6252531 , 36.728670

In [24]:
model = BayesianNetAtoms(net.functions, net.species, net.device)

pyro.render_model(model, model_args=([torch.ones(108, 52),torch.ones(108, 52)], [torch.ones([4, 108]),torch.ones([4, 108])], torch.ones(4,)))#,grp_energy))

NameError: name 'net' is not defined

In [ ]:
# 4. Initialize dataloader
grouped_train_loader = DataLoader(grouped_train_data, batch_size=1, shuffle=False,
                                  collate_fn=custom_collate, num_workers=0)
grouped_valid_loader = DataLoader(grouped_valid_data, batch_size=1, shuffle=False,
                                  collate_fn=custom_collate, num_workers=0)

In [ ]:
for data_batch in grouped_train_loader:
    grp_descrp, grp_energy, logic_reduce = data_batch[0][10], data_batch[0][11], data_batch[0][12]

In [ ]:
grp_energy

tensor([-27.0000, -17.8317,   2.4570,   3.1589], dtype=torch.float64)

In [ ]:
logic_reduce[0].shape

torch.Size([4, 108])

In [ ]:
grp_descrp[0] = grp_descrp[0].float()
logic_reduce[0] =logic_reduce[0].float()

In [ ]:
model.forward(grp_descrp, logic_reduce, grp_energy)

tensor([ 302.2620, -196.0373,  209.5111,  309.6779], grad_fn=<AddBackward0>)

In [ ]:
model.svi.step(grp_descrp, logic_reduce,grp_energy)

NotImplementedError: 
Trace Shapes:
 Param Sites:
Sample Sites:

In [ ]:
for m in model.modules():
    m.model(data_batch[0][10][0], data_batch[0][10][0], data_batch[0][12][0])

NotImplementedError: 

In [ ]:
model.model(data_batch[0][10], data_batch[0][12])

RuntimeError: expected scalar type Float but found Double

In [ ]:
for data_batch in grouped_train_loader:
    grp_descrp, grp_energy, logic_reduce, grp_N_atom = data_batch[0][10], data_batch[0][11], data_batch[0][12], data_batch[0][14]


In [ ]:
grp_N_atom

tensor([27., 27., 27., 27.], dtype=torch.float64)

In [ ]:
grp_descrp[0].shape

torch.Size([108, 52])

In [ ]:
for data_batch in grouped_train_loader:
    model.step(data_batch[0][10], data_batch[0][12])

RuntimeError: expected scalar type Float but found Double
                              Trace Shapes:        
                               Param Sites:        
                              Sample Sites:        
model.functions.0.Linear_Sp1_F1.weight dist | 15 52
                                      value | 15 52
  model.functions.0.Linear_Sp1_F1.bias dist | 15   
                                      value | 15   
Trace Shapes:
 Param Sites:
Sample Sites:

In [ ]:
data_batch[0][10]

[tensor([[-0.0789,  0.7212, -1.5706,  ...,  1.3901,  1.2308,  0.2393],
         [-1.0732,  0.9951,  0.0450,  ...,  0.4152, -0.1812,  0.9815],
         [ 0.3986, -0.4489,  0.1622,  ..., -0.2772, -0.7212,  0.4384],
         ...,
         [-0.6189, -2.1412,  1.5223,  ..., -1.7583, -1.2087, -2.4926],
         [ 1.6111,  0.4451, -0.4222,  ...,  0.3068,  0.3127, -0.5332],
         [-0.1283, -0.6618, -0.9102,  ...,  0.5214, -0.0743,  0.6787]],
        dtype=torch.float64),
 tensor([], size=(0, 52), dtype=torch.float64)]